In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, Binarizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier

train_labels_path = 'data/train_labels.csv'
train_values_path = 'data/train_values.csv'

train_labels = pd.read_csv(train_labels_path)
train_values = pd.read_csv(train_values_path)
data = train_values.merge(train_labels, on='building_id')

categorical_cols = [
    'land_surface_condition', 'foundation_type', 'roof_type',
    'ground_floor_type', 'other_floor_type', 'position',
    'plan_configuration', 'legal_ownership_status'
]

integer_cols = [
    'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
    'count_floors_pre_eq', 'age', 'area_percentage',
    'height_percentage', 'count_families'
]

binary_cols = [
    'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
    'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
    'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick',
    'has_superstructure_timber', 'has_superstructure_bamboo',
    'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered'
]

data.drop(columns=['count_families', 'legal_ownership_status'])


X = data.set_index('building_id').drop(['damage_grade'], axis=1)
y = data.set_index('building_id')['damage_grade']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('int', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), integer_cols),
        ('cat', OrdinalEncoder(handle_unknown='error'), categorical_cols),
        ('bin', Binarizer(), binary_cols)
    ],
    remainder='passthrough', verbose_feature_names_out=False
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

class FuzzyRandomForest(RandomForestClassifier):
    def __init__(self, n_estimators=400, fuzzy_param=1, random_state=None, min_samples_split=2, verbose=2,
                 max_features=None, max_depth=None, min_samples_leaf=1, max_leaf_nodes=None, class_weight=None):
        super().__init__(
            n_estimators=n_estimators,
            random_state=random_state,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf,
            min_samples_split=min_samples_split,
            max_features=max_features,
            max_leaf_nodes=max_leaf_nodes,
            verbose=verbose,
            class_weight=class_weight
        )
        self.fuzzy_param = fuzzy_param

    def predict_proba_fuzzy(self, X):
        proba = super().predict_proba(X)
        fuzzy_proba = proba ** self.fuzzy_param
        fuzzy_proba /= fuzzy_proba.sum(axis=1, keepdims=True)
        return fuzzy_proba

    def predict_fuzzy(self, X):
        fuzzy_proba = self.predict_proba_fuzzy(X)
        return np.argmax(fuzzy_proba, axis=1) + 1

# Micro-F1を評価指標として設定
micro_f1_scorer = make_scorer(f1_score, average='micro')

# Skip gridSearchCV because of exhaustive search (longer runtime)
#param_grid = {
#    'max_depth': [None, 10, 15],
#    'min_samples_leaf': [1, 3, 5],
#    'fuzzy_param': [1, 1.2, 1.5],
#    'class_weight': [None, 'balanced_subsample']
#}

#grid_search = GridSearchCV(
#    estimator=frf,
#    param_grid=param_grid,
#    scoring=micro_f1_scorer,
#    cv=3,
#    n_jobs=-1
#)
#best_model = grid_search.best_estimator_
#print("Best Params:", grid_search.best_params_)
#print("Best Micro-F1 (CV):", grid_search.best_score_)

frf = FuzzyRandomForest(
    n_estimators = 650,
    max_features = 25,
    min_samples_split = 17,
    random_state=42,
    )

frf.fit(X_train_processed, y_train)

y_pred_fuzzy = frf.predict_fuzzy(X_test_processed)
micro_f1_score_val = f1_score(y_test, y_pred_fuzzy, average='micro')

print("Test Micro-F1:", micro_f1_score_val)


building tree 1 of 650
building tree 2 of 650
building tree 3 of 650
building tree 4 of 650
building tree 5 of 650
building tree 6 of 650
building tree 7 of 650
building tree 8 of 650
building tree 9 of 650
building tree 10 of 650
building tree 11 of 650
building tree 12 of 650
building tree 13 of 650
building tree 14 of 650
building tree 15 of 650
building tree 16 of 650


KeyboardInterrupt: 